<a href="https://colab.research.google.com/github/masoupas/IA-3/blob/main/AlphaPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose Detection with AlphaPhose

This notebook uses an open source project [MVIG-SJTU/AlphaPose](https://github.com/MVIG-SJTU/AlphaPose) to detect/track multi person poses on a given video imported from Google Drive.


## Install AlphaPose

In [ ]:
! pip install pyyaml==5.2
! pip install scipy==1.1.0
! pip install torch==1.2.0 torchvision==0.4.0
! pip install pillow==6.2.2

     |████████████████████████████████| 266kB 9.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.2-cp37-cp37m-linux_x86_64.whl size=44209 sha256=cfbd5cbfdb71dd0595145d66f3c2298855e74d1b2e8026e418892b0851f1e41e
  Stored in directory: /root/.cache/pip/wheels/54/b7/c7/2ada654ee54483c9329871665aaf4a6056c3ce36f29cf66e67
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 31.2MB 101kB/s 
ERROR: pymc3 3.11.2 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 74

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q -b pytorch --depth 1 $git_repo_url
  !cd $project_name && pip install -q -r requirements.txt
  !pip install -q youtube-dl visdom
  
import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

ERROR: Could not find a version that satisfies the requirement torch==0.4.0 (from -r requirements.txt (line 1)) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2, 0.4.1, 0.4.1.post2, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1)
ERROR: No matching distribution found for torch==0.4.0 (from -r requirements.txt (line 1))
     |████████████████████████████████| 1.9MB 7.5MB/s 
     |████████████████████████████████| 686kB 44.4MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 


## Download pretrained models

In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  

  
pretrained_model_path = join(project_name, 'models/sppe/duc_se.pth')
if not exists(pretrained_model_path):
  # download the pretrained model
  download_from_google_drive('1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW', pretrained_model_path)
  
yolo_pretrained_model_path = join(project_name, 'models/yolo/yolov3-spp.weights')
if not exists(yolo_pretrained_model_path):
  # download the YOLO weights
  download_from_google_drive('1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC', yolo_pretrained_model_path)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1426      0 --:--:-- --:--:-- --:--:--  1426
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  227M    0  227M    0     0   118M      0 --:--:--  0:00:01 --:--:--  169M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1387      0 --:--:-- --:--:-- --:--:--  1387
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  240M    0  240M    0     0   111M      0 --:--:--  0:00:02 --:--:--  190M


## Detect poses on a video

Instructions:
- Name the file you want to use source.mp4
- Place source.mp4 file in a map on your Google Drive, called 'Videos/in/'

The code below cuts the source video to 10 seconds and performs the pose detection. The resulting video can then be downloaded from the left panel.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# run AlpaPose on video from Google Drive
# clear previous files
!rm -rf AlphaPose_video.avi video.mp4
# cut the first 5 seconds and copy video to AlphaPose directory
!ffmpeg -y -loglevel info -i "/content/drive/MyDrive/Colab Notebooks/PRACTICAS IDAL/20210428_135306.mp4"  "AlphaPose/video.mp4"
!cd $project_name && python3 video_demo.py --sp --video video.mp4 --outdir .. --save_video
# convert the result into MP4
!ffmpeg -y -loglevel info -i AlphaPose_video.avi AlphaPose_video.mp4
# copy result to Google Drive - disabled, too slow, advice manual download from left panel
#from shutil import copyfile
#copyfile(AlphaPose_video.mp4, "drive/My Drive/Videos/out/AlphaPose_video.mp4")

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib